In [15]:
!pip install kafka-python

In [18]:
import os 
packages = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1"

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    "--packages {0} pyspark-shell".format(packages)
)

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import to_json, struct, lit
sc = SparkContext('local')
spark = SparkSession(sc)

KAFKA_BROKER = "kafka:9092"
KAFKA_TOPIC = "hello"

#KAFKA_TOPIC="table1"

In [9]:
# df = spark.read.json("hdfs://namenode/user/root/data.json")
# dataSchema = df.schema

In [19]:

df=spark.read.format("json").option("mode", "FAILFAST")\
.option("inferSchema", "true").load("hdfs://namenode/user/root/data30001-60000.json")        
# df_list = df.collect()
# print(df_list[0])
# row_dict = df_list[0].asDict()
# print(row_dict)
df.show()


In [20]:
df.printSchema()

root
 |-- androidVersion: string (nullable = true)
 |-- category: string (nullable = true)
 |-- comments: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- contentRating: string (nullable = true)
 |-- currentVersion: string (nullable = true)
 |-- installs: long (nullable = true)
 |-- lastUpdate: long (nullable = true)
 |-- price: double (nullable = true)
 |-- ratings: long (nullable = true)
 |-- reviews: long (nullable = true)
 |-- score: double (nullable = true)
 |-- size: string (nullable = true)
 |-- title: string (nullable = true)



In [21]:
df = df.withColumn('value' ,to_json(struct([df[x] for x in df.columns])))
df.selectExpr("CAST(title AS STRING) as key", "CAST(value AS STRING) as valuess").show(10)

+--------------------+--------------------+
|                 key|             valuess|
+--------------------+--------------------+
|FeelingU-Live Vid...|{"androidVersion"...|
|Tick - Chat all o...|{"androidVersion"...|
|Vico - Video Chat...|{"androidVersion"...|
|Meet Strangers & ...|{"androidVersion"...|
|Vistar - Live Vid...|{"androidVersion"...|
|Peachat - Live Vi...|{"androidVersion"...|
|Meetbay - Live Ch...|{"androidVersion"...|
|Mico Cora - live ...|{"androidVersion"...|
|Group Voice Chat ...|{"androidVersion"...|
|Aussie Mingle - A...|{"androidVersion"...|
+--------------------+--------------------+
only showing top 10 rows



In [23]:
df.selectExpr("CAST(title AS STRING) as key", "CAST(value AS STRING)")\
  .write\
  .format("kafka")\
  .option("kafka.bootstrap.servers", KAFKA_BROKER)\
  .option("topic", KAFKA_TOPIC)\
  .save()

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".;

In [ ]:
df = df.withColumn('topic' ,lit(KAFKA_TOPIC))
df.selectExpr('topic', "CAST(title AS STRING) as key", "CAST(value AS STRING)").show(5)